# helper functions

In [41]:
# Importing Classiq libraries
from classiq import Model, synthesize, QReg, show
from classiq.builtin_functions import RZGate, RYGate, StatePreparation, CXGate, XGate, UnitaryGate

from classiq import Model, RegisterUserInput, ControlState

import numpy as np

In [42]:
# defining function for normalization (take over from razeen)

def normalize_complex_vector(alpha_prime, beta_prime):
    # Compute the magnitude of the vector
    magnitude = np.sqrt(alpha_prime * alpha_prime.conjugate() + beta_prime * beta_prime.conjugate())
    
    # Normalize the vector
    alpha = alpha_prime / magnitude
    beta = beta_prime / magnitude
    
    return alpha, beta

# definig function for angle a

def calculate_angle_difference(alpha, beta):
    # Compute the angle (phase) of alpha and beta
    angle_alpha = np.angle(alpha)
    angle_beta = np.angle(beta)
    
    # Calculate the difference in angles
    angle_difference = angle_alpha - angle_beta
    
    return angle_difference

# definig the function for finding w

def calculate_w(alpha, beta):
    # Calculate the magnitude of alpha and beta
    magnitude_alpha = np.abs(alpha)
    magnitude_beta = np.abs(beta)
    
    # Calculate w
    w = np.arctan(magnitude_alpha / magnitude_beta)
    
    # Check the condition and adjust w accordingly
    if 0 <= w <= np.pi/2 or np.pi <= w <= 3*np.pi/2:
        w = w
    elif np.pi/2 < w < np.pi or 3*np.pi/2 < w < 2*np.pi:
        w = -w
    
    return w

In [57]:
def g_gate(model, alpha, beta, dif_wire_number, dif, l, cx_out, n):
    # calculation from Shikhar
    alpha_prime = calculate_angle_difference(alpha, beta)
    omega = calculate_w(alpha, beta)

    G_matrix = [
        [np.sin(omega), np.exp(1j * alpha_prime) * np.cos(omega)],
        [np.exp(-1j * alpha_prime) * np.cos(omega), -np.sin(omega)]
    ]
    unitary_gate_params = UnitaryGate(data=G_matrix)

    cx1= CXGate()
    cx2= CXGate()

    # remember to have the qubits in the reverse order 

    if dif_wire_number<n:
        rz_params3 = RZGate(phi=alpha_prime)
        z3_out = model.RZGate(rz_params3, in_wires={"TARGET":cx_out[(n-1)-dif_wire_number]["CTRL"]})
        cx_ctrl_G1 = model.CXGate(cx1, in_wires={"TARGET": z3_out["TARGET"],"CTRL":t["OUT"][n-1]})
         #Rz(- a)
        rz_params1 = RZGate(phi=-alpha_prime)
        z1_out= model.RZGate(rz_params1, in_wires={"TARGET":cx_ctrl_G1["TARGET"]})
        # Ry(- w)
        ry_params1 = RYGate(theta=-omega)
        y1_out=model.RYGate(ry_params1, in_wires={"TARGET":z1_out["TARGET"]})
        # cx_gate
        cx_ctrl_G2 = model.CXGate(cx2, in_wires={"TARGET": y1_out["TARGET"],"CTRL":cx_ctrl_G1["CTRL"]})
        # Ry(w)
        ry_params2 = RYGate(theta=omega)
        y2_out=model.RYGate(ry_params2, in_wires={"TARGET":cx_ctrl_G2["TARGET"]})
        # Rz(a)
        rz_params2 = RZGate(phi=alpha_prime)
        z2_out= model.RZGate(rz_params2, in_wires={"TARGET":y2_out["TARGET"]})
    #G= model.UnitaryGate(unitary_gate_params, in_wires= {"TARGET":cx_out[dif_wire_number]["CTRL"]})
    if dif_wire_number>n and l==0:
        G = model.UnitaryGate(unitary_gate_params, in_wires= {"TARGET":t_out_dif["TARGET"]})
    if dif_wire_number>n and l==1:
        G = model.UnitaryGate(unitary_gate_params, in_wires= {"TARGET":t["OUT"][dif]})    


In [44]:
def find_qubit_with_unequal_sets(T):
    n = len(T[0])  # Number of qubits

    best_qubit = None
    Max_difference = float('-inf')  # Initialize to negative infinity

    for b in range(n):
        # Split T into T_0 and T_1 based on qubit b
        T_0 = [x for x in T if x[b] == 0]
        T_1 = [x for x in T if x[b] == 1]

        # Check if both sets are non-empty
        if len(T_0) != 0 and len(T_1) != 0:
            difference = abs(len(T_0) - len(T_1))
            if difference > Max_difference:
                Max_difference = difference
                best_qubit = b
                best_T_0 = T_0
                best_T_1 = T_1

    return best_qubit,best_T_0, best_T_1

# algorithm 1

In [93]:
def algorithm1(model, S, coefficients, num_qubits):
    # Step 1
    model = Model()
    # Step 2
    dif_qubits = []
    # Step 3
    dif_values = []
    # Step 4
    T = S
    # Finding qubit with maximum difference between set T_0 and T_1 
    # This is used in step 5
    P = find_qubit_with_unequal_sets(T)
    
    # Step 5: Main loop
    while len(T) > 1:
        # Step 6: Find the qubit b
        P = find_qubit_with_unequal_sets(T)  # We already implement this logic to find the best qubit best_T_0 and best_T_1 above
        b = P[0]
        T_0 = P[1]
        T_1 = P[2]
    
        # Step 7: Append b to dif_qubits
    
        dif_qubits.append(b)
    
        if len(T_0) < len(T_1):
            # Step 9: Set T = T_0 and append 0 to dif_values
            T = T_0
            dif_values.append(0)
        else:
            # Step 10: Set T = T_1 and append 1 to dif_values
            T = T_1
            dif_values.append(1)

    # Step 14: Pop the last value appended to dif_qubits and store it as dif
    dif = dif_qubits.pop();
    print("dif is  ", dif, "\n i.e. our 3rd qubit line will act as dif since numbering started from 0 ")
    # Step 15: Pop the last value that was appended to dif_values
    dif_values.pop();
    # Step 16: Store the single element in T as x_1
    x_1 = T[0]
    print("x1 is  ", x_1)
    # Step 17
    # T_prime subset of S denote the set of strings that have the values in dif_values on the bits dif_qubits
    T_prime = [x for x in S if all(x[q] == v for q, v in zip(dif_qubits, dif_values))]
    # Step 18: Remove x_1 from T'
    T_prime.remove(x_1)
    # Step 19: Second While loop for T_prime
    while len(T_prime) > 1:
        # Step 22: Find the qubit b_prime
        R = find_qubit_with_unequal_sets(T_prime)  # Implement logic to find the best qubit
        b_prime = R[0]
        T_prime_0 = R[1]
        T_prime_1 = R[2]
    
        # Step 21: Append b to dif_qubits
    
        dif_qubits.append(b_prime)
    
        if len(T_prime_0) < len(T_prime_1):
            # Step 23: Set T = T_0 and append 0 to dif_values
            T_prime = T_prime_0
            dif_values.append(0)
        else:
            # Step 25: Set T = T_1 and append 1 to dif_values
            T_prime = T_prime_1
            dif_values.append(1)
    # Step 28
    x_2 = T_prime[0]
    print("x2 is ", x_2)
    
    model = Model()
    probabilities = [1/5, 1/5, 1/5, 0, 1/5, 0, 0, 1/5] # big endian
    sp = StatePreparation(
        probabilities=probabilities, error_metric={"KL": {"upper_bound": 0.001}}
    )
    t=model.StatePreparation(params=sp)

    
    x_params= XGate()
    cx_params = ["cx1", "cx2", "cx3"]
    cx_out = ["co1", "co2", "co3"]
    l = 1
    l1 = [1] * num_qubits
    u = 0
    # u will be used for numbering of CX Gates
    if x_1[dif] != 1:
        t_out_dif = model.XGate(x_params, in_wires={"TARGET":t["OUT"][2-dif]})
        l=0        # if l=0 then dif wire is t_out["TARGET"] otherwise t["OUT"][dif]
    # Step 32: For qubit b in {1, 2, ..., n} \ dif
    count = [1] * num_qubits
    for b in range(num_qubits):
        if b != dif and x_1[b] != x_2[b]:
            count[b] = 0
            cx_params[u] = CXGate()
            l1[b]=0
            # Step 34: Add a CNOT gate targeting b controlled on dif
            if l==0:
                cx_out[2-b]=model.CXGate(cx_params[u],in_wires={"TARGET":t["OUT"][2-b],"CTRL":t_out_dif["TARGET"]})       
            else:
                cx_out[2-b]=model.CXGate(cx_params[u],in_wires={"TARGET":t["OUT"][2-b],"CTRL":t["OUT"][2-dif]})
            u=u+1
  
    x_last = ["x1", "x2", "x3"]
    x_last_out = [ "xl1", "xl2", "xl3"]              
    # Step 37: For qubit b in dif_qubits
    v=0
    # v will be used for numbering of CX Gates
    for b in dif_qubits:
        if x_2[b] != 1:
            x_last[v] = XGate()
            # Step 39: Add a NOT gate on line b
            if l1[b]==0:
                x_last_out[b]=model.XGate(x_last[v],in_wires={"TARGET":cx_out[2-b]["TARGET"]})
            else:
                x_last_out[b]=model.XGate(x_last[v],in_wires={"TARGET":t["OUT"][2-b]})
            v=v+1
            
    dif_wire_number=-1
    for b in reversed(range(num_qubits)):
        if l1[b]==0:
            dif_wire_number = b
            break
     
    # Step 42: Apply on qubit dif a G-gate controlled on the qubits in dif_qubits
    # calculate alpha
    # get c_x1
    idx_x1 = S.index(x_1)
    c_x1 = coefficients[idx_x1]
    # get c_x2
    idx_x2 = S.index(x_2)
    c_x2 = coefficients[idx_x2]

    coefficients_ = coefficients
    
    if x_1[b] == 0:
        merge_elem = x_1
        s_= S
        s_.remove(x_2)
        coefficients_[idx_x1] = c_x1+c_x2
        del coefficients_[idx_x2]
    else:
        merge_elem = x_2
        s_= S
        s_.remove(x_1)
        coefficients_[idx_x2] = c_x1+c_x2
        del coefficients_[idx_x1]
    print("applying G: on ", dif, " controlled by ", dif_qubits, " ", c_x1, " ", c_x2, " merging to: ", merge_elem)
    # normalize
    c_x1_ = np.sqrt(c_x1**2/(c_x1**2 + c_x2**2))
    c_x2_ = np.sqrt(c_x2**2/(c_x1**2 + c_x2**2))
    
    alpha = c_x1_
    beta = c_x2_
    
    
    # apply g gate   
    #g_gate(model, c_x1_, c_x2_, dif_wire_number, dif, l, cx_out, num_qubits);
    # calculation from Shikhar
    alpha_prime = calculate_angle_difference(alpha, beta)
    omega = calculate_w(alpha, beta)

    G_matrix = [
        [np.sin(omega), np.exp(1j * alpha_prime) * np.cos(omega)],
        [np.exp(-1j * alpha_prime) * np.cos(omega), -np.sin(omega)]
    ]
    unitary_gate_params = UnitaryGate(data=G_matrix)

    cx1= CXGate()
    cx2= CXGate()

    # remember to have the qubits in the reverse order 

    if dif_wire_number<num_qubits:
        rz_params3 = RZGate(phi=alpha_prime)
        z3_out = model.RZGate(rz_params3, in_wires={"TARGET":cx_out[(num_qubits-1)-dif_wire_number]["CTRL"]})
        cx_ctrl_G1 = model.CXGate(cx1, in_wires={"TARGET": z3_out["TARGET"],"CTRL":t["OUT"][num_qubits-1]})
         #Rz(- a)
        rz_params1 = RZGate(phi=-alpha_prime)
        z1_out= model.RZGate(rz_params1, in_wires={"TARGET":cx_ctrl_G1["TARGET"]})
        # Ry(- w)
        ry_params1 = RYGate(theta=-omega)
        y1_out=model.RYGate(ry_params1, in_wires={"TARGET":z1_out["TARGET"]})
        # cx_gate
        cx_ctrl_G2 = model.CXGate(cx2, in_wires={"TARGET": y1_out["TARGET"],"CTRL":cx_ctrl_G1["CTRL"]})
        # Ry(w)
        ry_params2 = RYGate(theta=omega)
        y2_out=model.RYGate(ry_params2, in_wires={"TARGET":cx_ctrl_G2["TARGET"]})
        # Rz(a)
        rz_params2 = RZGate(phi=alpha_prime)
        z2_out= model.RZGate(rz_params2, in_wires={"TARGET":y2_out["TARGET"]})
    #G= model.UnitaryGate(unitary_gate_params, in_wires= {"TARGET":cx_out[dif_wire_number]["CTRL"]})
    if dif_wire_number>num_qubits and l==0:
        G = model.UnitaryGate(unitary_gate_params, in_wires= {"TARGET":t_out_dif["TARGET"]})
    if dif_wire_number>num_qubits and l==1:
        G = model.UnitaryGate(unitary_gate_params, in_wires= {"TARGET":t["OUT"][dif]})    
    
    quantum_program = synthesize(model.get_model())
    show(quantum_program)
    
    return model, s_, coefficients_

# algorithm 2

In [ ]:
def algorithm2(s, coefficients, num_qubits):
    #first sanity check
    if num_qubits == 0 or len(s) > 2**num_qubits or len(coefficients) != len(s):
        print("Error: params wrong")
        return
    # check if coefficients fit:
    check_sum = 0.0
    for coeff in coefficients:
        check_sum += coeff**2
    if np.isclose(check_sum, 1.0, rtol=1e-05, atol=1e-08, equal_nan=False) == False:
        print("Error: Coefficients do not sum up to 1.0: ", check_sum)
        return
    C = QuantumCircuit(num_qubits, num_qubits)
    #phi = ""
    while len(s) > 1:
        C_hat, s_, coefficients_ = algorithm1(s, coefficients, num_qubits)
        C = C.compose(C_hat)
        # merge coefficients
        print("coefficients ", coefficients_)
        coefficients = coefficients_
        s = s_
        print(coefficients)
        print(s)
    
    # add not gates
    for i in range(len(s[0])):
        if s[0][i] == 1:
            C.x(i)
    #invert
    C = C.inverse()
    
    return C

# examples

## test algo 1 - merging one step

In [94]:
# Define the input set S for quantum state  
S = [[0, 0, 0], [0, 0, 1], [0, 1, 0],[1, 0, 0],[1, 1,1 ]] # Replace this with your quantum state basis elements list
coefficients = [1/np.sqrt(5), 1/np.sqrt(5), 1/np.sqrt(5), 1/np.sqrt(5), 1/np.sqrt(5)]  # Replace this with your actual coefficients list

In [95]:
model = Model()

In [96]:
#probabilities = [1/5, 1/5, 1/5, 0, 1/5, 0, 0, 1/5]# little endian
probabilities = [1/5, 1/5, 1/5, 0, 1/5, 0, 0, 1/5] # big endian

In [97]:
sp = StatePreparation(
    probabilities=probabilities, error_metric={"KL": {"upper_bound": 0.001}}
)
t=model.StatePreparation(params=sp)


In [98]:
model_, s_, c_ = algorithm1(model, S, coefficients, 3)
print(s_)
print(c_)

dif is   1 
 i.e. our 3rd qubit line will act as dif since numbering started from 0 
x1 is   [1, 1, 1]
x2 is  [1, 0, 0]
applying G: on  1  controlled by  [0]   0.4472135954999579   0.4472135954999579  merging to:  [1, 0, 0]
Opening: https://platform.classiq.io/circuit/cb4f67f0-3ecd-4a2b-9ab4-b7d9660b9951?version=0.31.0
[[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0]]
[0.4472135954999579, 0.4472135954999579, 0.4472135954999579, 0.8944271909999159]


In [100]:
quantum_program = synthesize(model_.get_model())
show(quantum_program)

Opening: https://platform.classiq.io/circuit/3743040d-cebe-4b90-a358-eb25f5ef890c?version=0.31.0
